Импортирую нужные библиотеки.

In [13]:
import os
import gc
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, pipeline
from trl import DPOConfig, DPOTrainer
from datasets import Dataset
from sklearn.model_selection import train_test_split
import bitsandbytes as bnb
import pandas as pd
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from safetensors.torch import load_file

Скачиваю данные, фильтрую их и разделяю на тренировочную и тестовую выборки.

In [2]:
splits = {
    "train": "data/train-00000-of-00001-b42a775f407cee45.parquet",
    "validation": "data/validation-00000-of-00001-134b8fd0c89408b6.parquet",
}
df_train = pd.read_parquet("hf://datasets/OpenAssistant/oasst1/" + splits["train"])
df_validation = pd.read_parquet(
    "hf://datasets/OpenAssistant/oasst1/" + splits["validation"]
)

frames = [df_train, df_validation]
df = pd.concat(frames)
df_filtered = df[(df["parent_id"].isnull()) & (df["lang"] == "ru")]["text"]
df_train, df_test = train_test_split(df_filtered, test_size=0.15, random_state=42)

In [4]:
df_train.head()

83546    Напиши список лучших программ для фотограмметр...
73030                                      Что такое Web3?
64915    Здравствуйте. Расскажите мне кратко, пожалуйст...
39827                    Как музыка влияет на наши эмоции?
12502    Какую роль играют алгоритмы в машинном обучени...
Name: text, dtype: object

In [5]:
df_test.head()

44660    Что можешь посоветовать, если никак не получае...
37502    Я хочу выращивать помидоры на гидропонике. Мож...
13754                             Как правильно свататься?
60617    Как ты можешь применить технологии глубокого о...
14725    Какой, на твой взгляд, лучший хорор фильм\сери...
Name: text, dtype: object

Использую модель Qwen/Qwen2.5-0.5B-Instruct, которая имеет небольшое количество параметров. Эта модель доступна на Hugging Face без авторизации, в отличие от других моделей. 

Задаю 4 одинаковых вопроса и прошу модель ответить кратко и осмысленно. Затем я формирую новую таблицу df_with_answers, содержащую вопросы и ответы. 

In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype="auto", device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

number_of_quetions = 4
df_with_answers = pd.DataFrame(columns=["text", "answer"])

for question in df_train:
    for i in range(number_of_quetions):

        messages = [
            {
                "role": "user",
                "content": "Ответь на вопрос кратко, осмысленно и полезно, примерно 3 предложения на русском языке:"
                + question,
            }
        ]

        text = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )

        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(**model_inputs, max_new_tokens=512)
        generated_ids = [
            output_ids[len(input_ids) :]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

        df_with_answers = pd.concat(
            [df_with_answers, pd.DataFrame({"text": [question], "answer": [response]})],
            ignore_index=False,
        )

In [9]:
df_with_answers.head()

,text,answer
0,Напиши список лучших программ для фотограмметр...,1. **Лучшие программы для фотограмметрии**: \...
1,Напиши список лучших программ для фотограмметр...,1. **Лучшие программы для фотограмметрии:** Ad...
2,Напиши список лучших программ для фотограмметр...,1. **Лучшие программы для фотограмметрии**: \n...
3,Напиши список лучших программ для фотограмметр...,1. **Инструменты для фотограммирования**: Пожа...
4,Что такое Web3?,"Web3 - это новая версия сети интернет, основан..."


Использую модель для оценивания ответов по десятибалльной шкале.

In [ ]:
texts = df_with_answers["text"].tolist()
answers = df_with_answers["answer"].tolist()

df_with_answers_and_grade = pd.DataFrame(columns=["text", "answer", "grade"])

for question, answer in zip(texts, answers):

    messages = [
        {
            "role": "user",
            "content": "Оцени данный ответ:"
            + answer
            + " на данный вопрос:"
            + question
            + " по десятибалльной шкале, основные критерии это краткость, полезность , осмысленность, использование русского языка, в ответ не пиши текст, в ответ напиши только одно число",
        }
    ]

    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=512)
    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    df_with_answers_and_grade = pd.concat(
        [
            df_with_answers_and_grade,
            pd.DataFrame({"text": [question], "answer": [answer], "grade": [response]}),
        ],
        ignore_index=False,
    )

In [2]:
df_with_answers_and_grade.head()

,text,answer,grade
0,Напиши список лучших программ для фотограмметр...,1. **Лучшие программы для фотограмметрии**: \...,7
1,Напиши список лучших программ для фотограмметр...,1. **Лучшие программы для фотограмметрии:** Ad...,7
2,Напиши список лучших программ для фотограмметр...,1. **Лучшие программы для фотограмметрии**: \n...,1. 3\n2. 4
3,Напиши список лучших программ для фотограмметр...,1. **Инструменты для фотограммирования**: Пожа...,1. 10\n2. 9\n3. 8
4,Что такое Web3?,"Web3 - это новая версия сети интернет, основан...",1


В строках, где модель выдала не одно число, возьму последнее число в качестве оценки текста.

In [3]:
mask = df_with_answers_and_grade["grade"].str.len().apply(lambda x: x > 1)
df_with_answers_and_grade.loc[mask, "grade"] = df_with_answers_and_grade.loc[mask, "grade"].str[-1]
df_with_answers_and_grade["grade"].str.len().apply(lambda x: isinstance(x, str)).sum()

0

In [4]:
df_with_answers_and_grade.head()

,text,answer,grade
0,Напиши список лучших программ для фотограмметр...,1. **Лучшие программы для фотограмметрии**: \...,7
1,Напиши список лучших программ для фотограмметр...,1. **Лучшие программы для фотограмметрии:** Ad...,7
2,Напиши список лучших программ для фотограмметр...,1. **Лучшие программы для фотограмметрии**: \n...,4
3,Напиши список лучших программ для фотограмметр...,1. **Инструменты для фотограммирования**: Пожа...,8
4,Что такое Web3?,"Web3 - это новая версия сети интернет, основан...",1


Формирую датасет для DPO-тренировки модели. Отранжирую оценку по каждому вопросу и выберу 2 лучших ответа в качестве положительных, а 2 худших ответа – в качестве отрицательных.

In [5]:
dpo_dataset_dict = {"prompt": [], "chosen": [], "rejected": []}

for question, group in df_with_answers_and_grade.groupby("text"):

    sorted_group = group.sort_values(by="grade", ascending=False)
    best_answers = sorted_group.head(2)
    worst_answers = sorted_group.tail(2)

    dpo_dataset_dict["prompt"].append(question)
    dpo_dataset_dict["prompt"].append(question)

    dpo_dataset_dict["chosen"].append(best_answers["answer"].iloc[0])
    dpo_dataset_dict["chosen"].append(best_answers["answer"].iloc[1])

    dpo_dataset_dict["rejected"].append(worst_answers["answer"].iloc[0])
    dpo_dataset_dict["rejected"].append(worst_answers["answer"].iloc[1])

dpo_dataset_dict = Dataset.from_dict(dpo_dataset_dict)

In [6]:
dpo_dataset_dict[1]

{'prompt': '"ничего" это',
 'chosen': '"Ничего" - это слово, которое используется для означения отсутствия или бездействия. Например: "И не могу мне ничего помочь!"',
 'rejected': 'Ничего, ничего.'}

Делаю fine-tune модели с помощью LoRA.

In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "k_proj",
        "gate_proj",
        "v_proj",
        "up_proj",
        "q_proj",
        "o_proj",
        "down_proj",
    ],
)

model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype="auto", device_map="auto"
)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name)

training_args = DPOConfig(
    max_steps=200,
    output_dir="/home/jovyan/stepanov/DPO_trained_llm",
    beta=0.1,
)

dpo_trainer = DPOTrainer(
    model,
    args=training_args,
    train_dataset=dpo_dataset_dict,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

dpo_trainer.train()
dpo_trainer.model.save_pretrained("/home/jovyan/stepanov/final_checkpoint_alligment")
tokenizer.save_pretrained("/home/jovyan/stepanov/final_checkpoint_tokenizer_alligment")

In [12]:
base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct", torch_dtype="auto", device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

model_dpo = PeftModel.from_pretrained(
    base_model, "/home/jovyan/stepanov/final_checkpoint_alligment"
)
model_dpo = model_dpo.merge_and_unload()

Оценим работу нашей модели, используя ту же языковую модель. Она выберет лучший ответ из двух предложенных, а мы дополнительно проверим результаты самостоятельно.

In [34]:
dpo_model_counter = 0
base_model_counter = 0

df_for_clustering = pd.DataFrame(columns=["text", "base_model_answer", "dpo_model_answer"])

for question in df_test:

    messages = [
        {
            "role": "user",
            "content": "Ответь на вопрос кратко, осмысленно и полезно, примерно 3 предложения на русском языке:"
            + question,
        }
    ]

    text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(base_model.device)
    model_inputs_dpo = tokenizer([text], return_tensors="pt").to(model_dpo.device)

    generated_ids = base_model.generate(**model_inputs, max_new_tokens=512)
    generated_ids_dpo = model_dpo.generate(**model_inputs, max_new_tokens=512)

    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    generated_ids_dpo = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(model_inputs_dpo.input_ids, generated_ids_dpo)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    response_dpo = tokenizer.batch_decode(generated_ids_dpo, skip_special_tokens=True)[0]

    df_for_clustering = pd.concat(
        [
            df_for_clustering,
            pd.DataFrame(
                {
                    "text": [question],
                    "base_model_answer": [response],
                    "dpo_model_answer": [response_dpo],
                }
            ),
        ],
        ignore_index=False,
    )

    message_for_clustering = [
        {
            "role": "user",
            "content": "Какой ответ на вопрос:" + question +  "лучше? Первый: "
            + response
            + " или второй:"
            + response_dpo
            + " напиши в ответе ОДНО число, 1 или 2"
        }
    ]

    text = tokenizer.apply_chat_template(
        message_for_clustering, tokenize=False, add_generation_prompt=True
    )

    model_inputs_for_clustering = tokenizer([text], return_tensors="pt").to(
        base_model.device
    )

    generated_ids_for_clustering = base_model.generate(
        **model_inputs_for_clustering, max_new_tokens=512
    )

    generated_ids_for_clustering = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(
            model_inputs_for_clustering.input_ids, generated_ids_for_clustering
        )
    ]

    response_for_clustering = tokenizer.batch_decode(
        generated_ids_for_clustering, skip_special_tokens=True
    )[0]

    if str(response_for_clustering)[0] == "1":
        base_model_counter += 1
    elif str(response_for_clustering)[0] == "2":
        dpo_model_counter += 1

print(base_model_counter, dpo_model_counter)

58 55


Количество выигрышей базовой модели: 58

Количество выигрышей дообученной модели: 55

Видим, что в среднем модель одинаково оценивает ответы на вопросы.

In [20]:
for j in range(5):
    i = random.randint(1, len(df_for_clustering))
    print(f"Вопрос: {df_for_clustering['text'][i]}")
    print("")
    print(f"Ответ базовой модели: {df_for_clustering['base_model_answer'][i]}")
    print("")
    print(f"Ответ тренированной модели: {df_for_clustering['dpo_model_answer'][i]}")
    print("")
    print("")
    print("")

Вопрос: Чем вместе занимались Карл, Маркс, Фридрих и Энгельс? Почему их упоминают вместе?

Ответ базовой модели: Карл Вольфганг Лейбниц, Маркс Павел Платонович, Фридрих Николаевич Берлинский, Энгельс Густав Иоганн фон

Ответ тренированной модели: Карл Вольфрам Лейбница, Марк Сальваторо Моррисон, Фридрих Николас Берлин и Энгельс Готфрид Ганнингс, в общем-то, не занимались этим вопросом. Они были лишь членами одной группы людей, занимающейся философским и политическим вопросам. Однако, они часто публично упоминались вместе как часть одного из самых значимых концепций экономической теории.



Вопрос: Расскажи о смутном времени говоря как некомпетентный человек и в дворовом стиле с матами. Не бойся использовать оскорбительные слова, я тебя прощаю xD

Ответ базовой модели: Вот краткое ответ:

Смутное время - это нечто загадочное, но не безумное. В дворовых стилях можно было бы увидеть тихие тени, которые могли быть неожиданными событиями или мечтами. Страна, где этот период произошел, была 

### Вывод:
В моем эксперименте дообучение не оказало заметного влияния на вывод модели. Это связано с несколькими факторами:

1. Недостаточное количество данных в наборе даже для такой небольшой модели.
2. Недостаточное качество сгенерированных ответов.
3. Возможно, выбран некорректный метод дообучения.

К процессу "выравнивания" нужно подходить серьезно, используя качественные синтетические данные и данные, аннотированные человеком. Также необходима мощная модель для ранжирования и оценки ответов, чтобы затем обучать модель метрическими методами.